# RDD

## 저수준 API란
- 스파크에는 두 종류의 저수준 API가 있음
  1. 분산 데이터 처리를 위한 RDD
  2. 분산형 공유 변수를 배포하고 다루기 위한 API(브로드캐스트 변수와 어큐뮬레이터)

### 저수준 API는 언제 사용할까
- 고수준 API에서 제공하지 않는 기능이 필요한 경우
  - 클러스터의 물리적 데이터의 배치를 아주 세밀하게 제어해야하는 상황
  - RDD를 사용해 개발된 기존 코드를 유지해야 하는 상황
  - 사용자가 정의한 공유 변수를 다뤄야 하는 상황

- 스파크의 모든 워크로드는 저수준 기능을 사용하는 기초적인 형태로 컴파일되므로 이를 이해하는 것은 많은 도움이 될 수 있음
  - DataFrame 트랜스포메이션을 호출하면 다수의 RDD 트랜스포메이션으로 변환

### 저수준 API는 어떻게 사용할까
- SparkContext는 저수준 API 기능을 사용하기 위한 진입 지점
- 스파크 클러스터에서 연산을 수행하는데 필요한 도구인 SparkSession을 이용해 접근 가능

In [0]:
spark.sparkContext

Out[1]:

SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 local[8] 
 AppName 
 Databricks Shell

## RDD 개요
- 불변성을 가지며 병렬로 처리할 수 있는 파티셔닝된 레코드의 모음

- RDD의 레코드는 프로그래머가 선택하는 자바, 스칼라, 파이썬의 객체
  - DataFrame의 레코드는 스키마를 알고, 필드로 구성된 구조화된 로우

- 이러한 객체에는 사용자가 원하는 포맷을 사용해 원하는 모든 데이터를 저장할 수 있음

- 모든 값을 다루거나 값 사이의 상호작용 과정은 반드시 수동으로 정의

- spark에서는 RDD레코드의 내부 구조를 파악할 수 없으므로 수작업으로 최적화
  - 필터 재정렬, 집계 등의 최적화 기법 직접 구현

### RDD 유형
- 두 가지 타입의 RDD를 만들 수 있음
  1. 제네릭 RDD 타입
  2. 키-값 RDD 타입
  
- RDD의 주요 속성
  - 파티션의 목록
  - 각 조각을 연산하는 함수
  - 다른 RDD와의 의존성 목록
  - 부가적으로 키-값 RDD를 위한 Partitioner
  - 부가적으로 각 조각을 연산하기 위한 기본 위치 목록
  
- 이러한 속성은 사용자 프로그램을 스케줄링하고 실행하는 스파크의 모든 처리 방식을 결정

- 또한 RDD 역시 트랜스포메이션, 액션 제공
  - DataFrame과 Dataset의 트랜스포메이션, 액션과 동일한 방식으로 동작
    - 하지만 RDD에는 '로우'개념이 없으므로 구조적 API에서 제공하는 여러 함수를 사용못하므로 수동으로 처리

- 언어별 성능 차이
  - 스칼라, 자바는 비슷
  - 파이썬은 상당한 성능 저하가 발생
    - 오버헤드 발생: 직렬화 -> 파이썬 프로세스에 전달 -> 처리 -> 다시 직렬화 -> JVM에 반환
    - 따라서 구조적 API를 사용하는 것이 좋음

### RDD는 언제 사용할까
- 정말 필요한 경우가 아니라면 수동으로 RDD를 생성하면 X
- DataFrame이 RDD보다 더 효율적이고 안정적이고 표현력이 좋음
- 물리적으로 분산된 데이터(자체적으로 구성한 데이터 파티셔닝)에 세부적인 제어가 필요할 때 RDD를 사용하는 것이 가장 적합

### Dataset과 RDD의 케이스 클래스
- Dataset과 케이스 클래스를 
- Dataset은 구조적 API가 제공하는 풍부한 기능과 최적화 기법을 제공한다는 점이 RDD와의 큰 차이점
- Dataset을 사용하면 JVM 데이터 타입과 스파크 데이터 타입 중 어떤 것을 쓸지 고민하지 않아도 됨
  - 모두 성능 동일

## RDD 생성하기

### DataFrame, Dataset으로 RDD 생성하기
- rdd 메서드를 호출하면 쉽게 변환 가능

In [0]:
spark.range(10).rdd

Out[6]: MapPartitionsRDD[24] at javaToPython at NativeMethodAccessorImpl.java:0

In [0]:
#위에서 만들어진 데이터를 처리하려면 Row 객체를 올바른 데이터 타입으로 변환하거나 Row 객체에서 값을 추출해야함
spark.range(10).toDF('id').rdd.map(lambda row: row[0])

Out[5]: PythonRDD[18] at RDD at PythonRDD.scala:58

In [0]:
#RDD -> DataFrame
spark.range(10).rdd.toDF()

Out[7]: DataFrame[id: bigint]

### 로컬 컬렉션으로 RDD 생성하기
- sparkContext의 parallelize 메서드 호출
  - 단일 노드에 있는 컬렉션을 병렬 컬렉션으로 전환

In [0]:
myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(' ')
words = spark.sparkContext.parallelize(myCollection, 2) #파티션 수: 2

In [0]:
#RDD에 이름을 지정하면 스파크 UI에 지정한 이름으로 RDD가 표시됨
words.setName('myWords')
words.name()

Out[10]: 'myWords'

### 데이터소스로 RDD 생성하기
- sparkContext를 사용해 데이터를 RDD로 읽을 수 있음

In [0]:
#줄 단위로 텍스트 파일을 RDD로 읽음
spark.sparkContext.textFile('/FileStore/tables/withTextFiles')

Out[11]: /FileStore/tables/withTextFiles MapPartitionsRDD[38] at textFile at NativeMethodAccessorImpl.java:0

In [0]:
#여러 텍스트 파일의 각 줄을 레코드로 가진 RDD를 생성함
spark.sparkContext.wholeTextFiles('/FileStore/tables/withTextFiles')

## 트랜스포메이션

### distinct

In [0]:
#distinct
words.distinct().count()

Out[12]: 10

### filter

In [0]:
#조건 함수
def startsWithS(individual):
  return individual.startswith("S")

In [0]:
words.filter(lambda word: startsWithS(word)).collect()

Out[14]: ['Spark', 'Simple']

### map

In [0]:
words2=words.map(lambda word: (word, word[0], word.startswith("S")))

In [0]:
words2.filter(lambda record: record[2]).take(5)

Out[21]: [('Spark', 'S', True), ('Simple', 'S', True)]

### flatMap
- 단일 로우를 여러 로우로 변환

In [0]:
words.flatMap(lambda word: list(word)).take(5)

Out[23]: ['S', 'p', 'a', 'r', 'k']

### sortBy

In [0]:
words.sortBy(lambda word: len(word)*-1).take(5)

Out[24]: ['Definitive', 'Processing', 'Simple', 'Spark', 'Guide']

### randomSplit
- RDD를 임의로 분할해서 RDD 배열을 만들 때 사용

In [0]:
fiftyFiftySplit = words.randomSplit([0.5,0.5])

In [0]:
len(fiftyFiftySplit)

Out[31]: 2

In [0]:
fiftyFiftySplit[0].collect()

Out[30]: ['Definitive', 'Guide', ':', 'Big', 'Data', 'Processing']

In [0]:
fiftyFiftySplit[1].collect()

Out[32]: ['Spark', 'The', 'Made', 'Simple']

##액션

### reduce
- RDD의 모든 값을 하나의 값으로 만듦

In [0]:
spark.sparkContext.parallelize(range(1,21)).reduce(lambda x,y: x+y)

Out[33]: 210

In [0]:
def wordLengthReducer(leftWord, rightWord):
  if len(leftWord)>len(rightWord):
    return leftWord
  return rightWord

In [0]:
words.reduce(wordLengthReducer)

Out[36]: 'Processing'

---
- wordLengthReducer 함수는 두 개의 입력값을 하나의 결과로 만들기 때문에 reduce메서드를 설명하는데 적합
- 파티션에 대한 리듀스 연산은 비결정적인 특성을 가짐
- 따라서 reduce 메서드를 실행할 때마다 다른 결과를 반환할 수 있음

### count

In [0]:
words.count()

Out[47]: 10